In [1]:
# Misc imports
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")
import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
import yaml
from pathlib import Path
# Local imports
from ese.scripts.utils import get_option_product
from ese.experiment.analysis.run_inference import get_cal_stats
from ese.experiment.analysis.analysis_utils.submit_utils import get_ese_inference_configs
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
scratch_root = Path("/storage/vbutoi/scratch/ESE")
code_root = Path("/storage/vbutoi/projects/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

/storage/vbutoi/projects/ionpy/pandas/register.py:47: UserWarning: registration of accessor <class 'ionpy.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'fillNA' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  register_series_accessor(method.__name__)(AccessorMethod)
/storage/vbutoi/projects/ionpy/pandas/register.py:47: UserWarning: registration of accessor <class 'ionpy.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'isNA' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  register_series_accessor(method.__name__)(AccessorMethod)
/storage/vbutoi/projects/ionpy/pandas/register.py:47: UserWarning: registration of accessor <class 'ionpy.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'notNA' for type <class 'pandas.core.series.Series'> is overriding 

In [2]:
%%yaml inference_config


experiment:
    exp_root: '?'
    seed: 40
    threshold: 0.5 
    max_ensemble_samples: 5

log:
    root: '?'
    min_fg_pixels: 0 
    log_interval: 20 
    save_preds: False
    log_image_stats: True 
    log_pixel_stats: True 
    gether_inference_stats: True
    track_ensemble_member_scores: False 
    summary_compute_global_metrics: True

dataloader:
    batch_size: 1 
    num_workers: 0
    pin_memory: True 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibration_cfg 

calibrator:
    _name: '?'
    normalize: False

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3
    square_diff: False 

global_calibration:
    num_prob_bins: 15
    num_classes: 2
    neighborhood_width: 3
    square_diff: False 
    loss_weights: None 

<IPython.core.display.Javascript object>

In [4]:
%%yaml standard_model_cfg 

# For standard datasets
#####################################
data:
    splits: ('val', 'cal')
    preload: False 

ensemble:
    combine_fn: 'mean'
    combine_quantity: 'probs'
    normalize: False
    num_members: 5

model:
    _type: "standard"
    pretrained_exp_root : None
    checkpoint: '?'
    pretrained_select_metric: "val-dice_score"
    pred_label: 1

<IPython.core.display.Javascript object>

In [5]:
exp_name = '06_04_24_WMH_DifferenceExps'
group_dict = {
    "datasets": [
        "WMH"
    ],
    "exp_group": exp_name,
    "model_type": "standard",
    "base_models_group": "/storage/vbutoi/scratch/ESE/training/01_03_24_WMH_Ensemble",
    "calibrated_models_group": "/storage/vbutoi/scratch/ESE/calibration/January_2024/01_18_24_WMH_All_Calibrators",
    "scratch_root": "/storage/vbutoi/scratch/ESE",
    "inf_cfg_root": "/storage/vbutoi/projects/ESE/ese/experiment/configs/inference",
}

## Gather Inference Options.

In [6]:
%%yaml inference_config_opts

calibrator:
    - Uncalibrated
    - TempScaling
    - LTS

do_ensemble:
    - False

<IPython.core.display.Javascript object>

In [7]:
# Load the inference cfg from local.
##################################################
inf_cfg_root = code_root / "ese" / "experiment" / "configs" / "inference"

##################################################
with open(inf_cfg_root / "Calibration_Metrics.yaml", 'r') as file:
    cal_metrics_cfg = yaml.safe_load(file)

##################################################
base_cfg = Config(inference_config).update([calibration_cfg, cal_metrics_cfg, standard_model_cfg])

In [8]:
# Get the configs for the different runs.
inf_cfgs = get_ese_inference_configs(
    group_dict=group_dict,
    base_cfg=base_cfg,
    inf_cfg_opts=inference_config_opts
)

WARNING! Calibrator not found, using default calibrator name: Uncalibrated.


In [9]:
len(inf_cfgs)

12

## Running Jobs

In [10]:
from ese.experiment.analysis.run_inference import get_cal_stats

In [11]:
from ese.experiment.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=inf_cfgs[0], 
    job_func=get_cal_stats,
    run_name='debug',
    show_examples=True,
    gpu='0',
) 

Set seed: 43


/storage/vbutoi/projects/ionpy/util/libcheck.py:49: UserWarning: Intel MKL extensions not available for NumPy
  warn("Intel MKL extensions not available for NumPy")
/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")
/storage/vbutoi/projects/ionpy/util/libcheck.py:59: UserWarning: PyTorch cannot find a valid GPU device, check CUDA_VISIBLE_DEVICES
  warn("PyTorch cannot find a valid GPU device, check CUDA_VISIBLE_DEVICES")


RuntimeError: No CUDA GPUs are available

In [ ]:
# from ese.experiment.experiment import submit_ese_exps 

# #### Run Batch Jobs
# submit_ese_exps(
#     config_list=inf_cfgs,
#     job_func=get_cal_stats,
#     available_gpus=['0', '1', '2', '3'],
# )